# Erik's run script for the $Z_2$ gauge theory with Fermions

**Authors:**
- Clement Charles
- Erik Gustafson
- Elizabeth Hardt
- Florian Herren
- Norman Hogan
- Henry Lamm
- Sara Starecheski
- Ruth Van der Water
- Mike Wagman

The following code brings together all the necessary algorithms together to the simulations for our production run.

In [1]:
#Standard modules
import sys
import numpy as np
import yaml
import pandas
#Qiskit modules
from qiskit import IBMQ, QuantumCircuit, transpile, QuantumRegister, execute
from qiskit.providers.ibmq import RunnerResult
from qiskit import Aer
from qiskit.providers import aer
from qiskit.providers.aer import noise
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import Statevector
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.visualization import plot_histogram

import datetime
#Project modules
sys.path.append('./modules/')
sys.path.append('./inputs/')
sys.path.append('./plots/')
import StatePrep as sprep
import DD_PassManager as dd_pm
import paulitwirlingpass as pts
from Z2analysis import nsites2nqubits
import Trotterization as Trot

# for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import Image
plt.rcParams["figure.autolayout"] = True
colors = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline

#For testing reload modules after edits
from importlib import reload
%load_ext autoreload

C:\Users\gusta\AppData\Local\Temp\ipykernel_14700\1931160774.py:14: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


# Get input parameters from `yaml` file

In [2]:
### get run parameters from YAML input file ###

stream = open('inputs/Z2_input.yaml', 'r')
dict_in = yaml.safe_load(stream)
stream.close()

account_info = dict_in['account_info'] # dictionary of account information
backend_opts = dict_in['backend_opts']   # dictionary of machine/simulator choices
simulation_opts = dict_in['simulation_opts'] # dictionary of lattice-simulation parameters
runtime_opts = dict_in['runtime_opts'] # dictionary of runtime parameters

### transpilation parameters ###
initial_layout = simulation_opts['layout']['initial_layout']
### Lattice parameters ###
mass = simulation_opts['lattice_params']['mass']       #fermion mass in lattice-spacing units
nsites = simulation_opts['lattice_params']['nsites']   #must be even for staggered simulation
nqubits = nsites2nqubits(nsites)
print(nqubits, nsites)
### Time evolution ###
tf = simulation_opts['time_evolution']['tf']       # total time interval in lattice-spacing units
ntrotter = simulation_opts['time_evolution']['ntrotter'] # number of trotter steps
epsilon = tf/ntrotter                              # step size in lattice-spacing units

### Pauli Twirling ###
twirl = simulation_opts['pauli_twirling']['twirl']     # if True, apply Pauli twirling
ntwirls = simulation_opts['pauli_twirling']['ntwirls'] # number of Pauli twirls

### dynamic decoupling ###
dd_method = simulation_opts['dynamic_decoupling']['method'] # if not none apply a given dynamic decoupling method

### Richardson ??? ###
richardson_level = simulation_opts['richardson_level'] # number of CNOTs for Richardson smearing

11 6


# Initialize IBM Account

In [3]:
### initialize IBM Quantum account ###

IBMQ.load_account()
hub = 'ibm-q-research'#account_info['hub']
group = 'fermilab-1'#account_info['group']
project = 'main'#account_info['project']
provider = IBMQ.get_provider(hub=hub, group=group, project=project)

In [4]:
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-ncsu', group='nc-state', project='scattering-simul')>,
 <AccountProvider for IBMQ(hub='ibm-q-research', group='fermilab-1', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>]

# Choose target backend for simulations to run on

In [5]:
def get_backend(provider, machine, nqubits, simulator, sim_noise):
    
    ## Run on IBMQ qasm simulator
    if (simulator):
        backend = provider.get_backend('ibmq_qasm_simulator')
        
        if (sim_noise != "None"):
            device_backend = provider.get_backend(sim_noise)
            noise_model = noise.NoiseModel.from_backend(device_backend)
            backend.set_options(noise_model=noise_model.to_dict())
            
        print(f'Running on ibmq_qasm_simulator with {sim_noise} noise_model\n') 
        return backend
    ## Run on specified quantum computer(s)
    else:
#         devices = provider.backends(filters=lambda x: not (x.name() == 'ibmq_casablanca' or x.name() == 'ibmq_armonk')
#                                     and nqubits <= x.configuration().n_qubits <= 7
#                                     and not x.configuration().simulator
#                                     and x.status().operational == True)
#         print(type(devices))
#         print(type(devices[0]))
        #print(f'Backends matching critera ares {devices}\n')
        #print(device_names)
        
        if (machine == 'least_busy'):
            machine = str(least_busy(devices))
            backend = provider.get_backend(machine)    
        
        elif (machine[0:4] == 'ibmq_'):
            backend = provider.get_backend(machine)   
            
        else:
            backend = provider.get_backend(str(machine))
            
        print(f'Running on machine {machine}\n')
        return backend
        

In [6]:

# machine
machine = backend_opts['machine']           # machine name or "least busy"
simulator = backend_opts['qasm_simulator']  # if True, use ibmq_qasm_simulator 
noise_model = backend_opts['noise_model']   # None or device noise model 
print(noise_model)
backend = get_backend(provider, machine, nqubits, simulator, noise_model)

ibmq_belem
Running on machine ibmq_jakarta



# Assign Runtime Program (and inputs)

assigns values from `yaml` file

In [7]:
# number of shots per simulation
nshots = runtime_opts['nshots'] 

# Not really sure what this is needed for...
program_id = runtime_opts['program_id']

# correct for readout error using calibration matrix if True
measurement_error_mitigation = runtime_opts['readout_error_mitigation'] 

#save results for each shot if True
memory = runtime_opts['memory'] 

# Degree of layout optimization by transpiler
# 1 is least optimized; 3 is most optimized
optimization_level = runtime_opts['optimization_level'] 

In [8]:
nsites2nqubits(nsites), nsites

(11, 6)

In [9]:
def build_circuit(nq, nsteps, epsilon, mass, init_state=None,
                  dynamic_decoupling=False, Twirling=False):
    
#     print(nq)
    if init_state is None:
        if nq == 3:
            qc = QuantumCircuit(nq, nq)
            qc.h(0)
            qc.cx(0, 1)
            qc.cx(0, 2)
            qc.x(2)
            qc.h(1)
#             qc = sprep.prepare_states_nsites2(nq)['meson']
        elif nq == 7:
            qc = QuantumCircuit(nq, nq)
            qc.h(2)
            qc.cx(2, 3)
            qc.cx(3, 4)
            qc.x([2, 6])
            qc.h([1, 3, 5])
        elif nq == 11:
            qc = QuantumCircuit(nq, nq)
            qc.h(4)
            qc.cx(4, 5)
            qc.cx(5, 6)
            qc.x([2, 6, 10])
            qc.h([1, 3, 5, 7, 9])
        elif nq == 15:
            qc = QuantumCircuit(nq, nq)
            qc.h(6)
            qc.cx(6, 7)
            qc.cx(7, 8)
            qc.x([2, 6, 10, 14])
            qc.h([1, 3, 5, 7, 9, 11, 13])
            #             qc = sprep.prepare_states_nsites4(nq)['meson']

        qc = Trot.trotter_evolution_generic(qc, nsites, epsilon, mass, nsteps)
#         for i in range(4):
        qc.h([i for i in range(0, nq, 2)])
        qc.measure([i for i in range(nq)], [i for i in range(nq)])
        return qc
#         return transpile(qc.decompose(), basis_gates=['cx', 'rz', 'sx'])
    else:
        print('not implemented')
        

# assemble circuits to run

In [14]:
circuits_no_dd = []
circuits_w_dd = []
tvals = []
twirl = False
for i in range(0, ntrotter + 1 + 480):
    print(i)
    tvals.append(np.round(i * epsilon, 8))
    # add time evolution circuit
    qc = build_circuit(nqubits, i, epsilon, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        circuits_no_dd += qcs
    else:
        circuits_no_dd.append(qc)
    # add rescaling circuit
    qc = build_circuit(nqubits, i, 0, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level)
        circuits_no_dd += qcs
    else:
        circuits_no_dd.append(qc)
    
circuits = circuits_no_dd

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gusta\miniforge3\lib\site-packages\IPython\core\interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\gusta\AppData\Local\Temp\ipykernel_14700\2480272941.py", line 9, in <cell line: 5>
    qc = build_circuit(nqubits, i, epsilon, mass)
  File "C:\Users\gusta\AppData\Local\Temp\ipykernel_14700\1833356933.py", line 37, in build_circuit
    qc = Trot.trotter_evolution_generic(qc, nsites, epsilon, mass, nsteps)
  File "C:\Users\gusta\source\repos\z2_matter_qc\simulation_main\./modules\Trotterization.py", line 142, in trotter_evolution_generic
    qc = Z2gates.apply_gauge_terms(qc, nsites, epsilon)
  File "C:\Users\gusta\source\repos\z2_matter_qc\simulation_main\./modules\Z2gates.py", line 68, in apply_gauge_terms
    qc.rx(epsilon / 2, 2 * link + 1)
  File "C:\Users\gusta\miniforge3\lib\site-packages\qiskit\circuit\quantumcircuit.py", line 3020, in rx
    return self.append(RXGa

TypeError: object of type 'NoneType' has no len()

In [ ]:
res = execute(circuits[::2], optimization_level=0, backend=Aer.get_backend('qasm_simulator'), shots=1e5)

In [ ]:
obs = np.zeros(501)
for i in range(501):
    counts = res.result().get_counts(i)
    val = 0
    for key in counts.keys():
        if key[4:7].count('1') % 2 == 0:
            val += counts[key] / 1e5
        else:
            val -= counts[key] / 1e5
    obs[i] += val
obs

# build the runtime-object

In [35]:
program_inputs_list = []
for i in range(4):
    program_inputs = {'circuits': circuits[300 * i:300 * (i + 1)],
                      'optimization_level': 0,
                      'measurement_error_mitigation': measurement_error_mitigation,
                      'memory': memory,
                      'shots': nshots}
    program_inputs_list.append(program_inputs)

provider.runtime.run()

In [110]:

job1 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[0],
                           result_decoder=RunnerResult)

C:\Users\gusta\miniforge3\lib\site-packages\qiskit\providers\ibmq\runtime\utils.py:218: UserWarning: Callable <function <lambda> at 0x0000022599478040> is not JSON serializable and will be set to None.
  warnings.warn(f"Callable {obj} is not JSON serializable and will be set to None.")


In [ ]:

job2 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[1],
                           result_decoder=RunnerResult)

In [ ]:

job3 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[2],
                           result_decoder=RunnerResult)

In [ ]:

job4 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[3],
                           result_decoder=RunnerResult)

In [ ]:
jobs = [job1, job2, job3, job4]

In [ ]:
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare', 'counts mitigated'])
data_row = []
ntval = 0
for k in range(len(jobs)):
    for j in range(300):
        data_dict = {}
        data_dict['jobid'] = jobs[k].job_id()
        data_dict['machine'] = jobs[k].backend().name()
        data_dict['date'] = datetime.datetime.today()
        if (j // ntwirls) % 2 == 0:
            data_dict['sim type'] = 'evolution'
            data_dict['nt'] = ntval // (ntwirls * 2) + 1
            nt+=1
            data_dict['pauli twirling'] = twirl
            if k > 4:
                data_dict['dynamic decoupling'] = True
            else:
                data_dict['dynamic decoupling'] = False
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
            data_dict['counts mitigated'] = jobs[k].result().results[j].data.quasiprobabilities
            data_row.append(data_dict)
        else:
            data_dict['sim type'] = 'rescaling'
            data_dict['nt'] = ntval // (ntwirls * 2) + 1
            nt+=1
            data_dict['pauli twirling'] = twirl
            if k > 4:
                data_dict['dynamic decoupling'] = True
            else:
                data_dict['dynamic decoupling'] = False
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
            data_dict['counts mitigated'] = jobs[k].result().results[j].data.quasiprobabilities
            data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

In [ ]:
dataframe.to_csv(f'simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')